# Рисуем интерактивные карты с Folium

В питоне есть множество библиотек, с помощью которых можно рисовать и анализировать пространственную информацию (spatial analysis).

Вот некоторые из них:

* folium
* gmaps
* basemap
* cartopy
* geoplotlib


В этом туториале пойдет речь о библиотеке Folium (https://github.com/python-visualization/folium), которая представляет собой питон-обертку над JS библиотекой Leaflet.

Большим плюсом по сравнению с другими библиотеками является интерактивность. Карты можно зумить, исследовать, кликать на маркеры.
Минусом ялвяется производительность. Создание карты с большим количеством точек может составлять минуты.

"Manipulate your data in Python, then visualize it in on a Leaflet map via Folium." (c) github

К сожалению, информация по работе с библиотекой раскидана по разным сайтам и туториалам. В официальной документации информации очень мало. А в русскоязычном интернете вообще ничего нет.

## Подготовка данных

Будем исследовать датасет с пабами Москвы =). Идея нагло стырена отсюда:

http://www.math.uwaterloo.ca/tsp/pubs/

In [ ]:
import json
import pandas as pd
import folium

with open('pubs.json') as json_data:
    d = json.load(json_data)
    
columns = ['lat', 'lon', 'name_ru', 'opening_hours', 'website']
index = range(0, len(d["data"]))
pubs = pd.DataFrame(columns = columns, index = index)

for i in range(0, len(d["data"])):
    pubs['lat'].iloc[i] = d["data"][i]["lat"]
    pubs['lon'].iloc[i] = d["data"][i]["lon"]
    pubs['opening_hours'].iloc[i] = d["data"][i]["opening_hours"]
    pubs['website'].iloc[i] = d["data"][i]["website"]
    pubs['name_ru'].iloc[i] = d["data"][i]["name_ru"]    

In [46]:
pubs.head(3)

,lat,lon,name_ru,opening_hours,website
0,55.6051,37.3538,Московский дворик,None,None
1,55.9092,37.8661,Кружка,None,None
2,55.8649,37.6056,Кружка,None,http://www.kruzhka.ru/


In [48]:
# Для центрирования карт я выбрала центральную точку Москвы

kreml = [55.750730, 37.617322]

## Dot density map

Самый простой вариант анализа - отобразить данные как точки (или маркеры) на карте.
В popup (выноску) положим название заведения и часы работы.

Различные типы маркеров задаются фукнциями:

- RegularPolygonMarker
- CircleMarker
- PolygonMarker

Возможные атрибуты: color, fill_color, weight, radius, number_of_sides (для RegularPolygonMarker).

Так же в выноску можно передавать график vincent (https://github.com/wrobstory/vincent) с помощью синтаксиса: 

folium.Popup().add_child(folium.Vega())

In [108]:
pubs_map = folium.Map(location=kreml, zoom_start=11)

for i in range(0, len(pubs)):
    folium.Marker([pubs['lat'].iloc[i], pubs['lon'].iloc[i]], popup = str(pubs['name_ru'].iloc[i]) + ": " 
                  + str(pubs['opening_hours'].iloc[i])).add_to(pubs_map)
    
pubs_map

Cluster marker - раскраска в зависимости от плотности точек. Близкие точки сливаются в один маркер.

Судя по данным наибольшая плотность пабов в Москве на серево-востоке от Кремля в районе Чистых прудов.

In [110]:
from folium import features

pubs_map = folium.Map(location=kreml, zoom_start=12)
mc = features.MarkerCluster()

for i in range(0, len(pubs)):
    mk = features.Marker([pubs['lat'].iloc[i], pubs['lon'].iloc[i]])
    mc.add_child(mk)
    
pubs_map.add_child(mc)

## Линии

In [ ]:
pubs_map = folium.Map(location=kreml, zoom_start=12)